<a href="https://colab.research.google.com/github/cagBRT/SentimentTextAnalysis/blob/master/Text_CNN_interpretation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Can we view the words used to make predictions in a review?<br>
https://towardsdatascience.com/viewing-text-through-the-eyes-of-a-machine-db30c744ee17


In [1]:
# Clone the entire repo.
%cd /content/
!git clone  https://github.com/cagBRT/SentimentTextAnalysis.git cloned-repo
%cd cloned-repo
!ls

/content
Cloning into 'cloned-repo'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 239 (delta 47), reused 0 (delta 0), pack-reused 163
Receiving objects: 100% (239/239), 1.77 MiB | 2.69 MiB/s, done.
Resolving deltas: 100% (144/144), done.
/content/cloned-repo
amazon_cells_labelled.txt	 Sentiment_Text_Analysis_4.ipynb
coffeedata.csv			 Sentiment_Text_Analysis_5.ipynb
images				 Sentiment_Text_Analysis_6.ipynb
imdb_labelled.txt		 Sentiment_Text_Analysis_7.ipynb
README.md			 Sentiment_Text_Analysis_8.ipynb
readme.txt			 Sentiment_Text_Analysis.ipynb
Sentiment_Text_Analysis_1.ipynb  Text_CNN_interpretation.ipynb
Sentiment_Text_Analysis_2.ipynb  yelp_labelled.txt
Sentiment_Text_Analysis_3.ipynb


## CNN intepretation for text classification
Creating a heatmap identify words and phrases which correlate to model predictions

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython.display import HTML
from sklearn.model_selection import train_test_split
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from keras.models import Model
from keras import backend as K 
from keras.models import Sequential
from keras import layers
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
df = pd.read_csv('/content/cloned-repo/coffeedata.csv') #from https://drive.google.com/open?id=17jYWjTHrwMegjdTYs0YLXyc9PDRLOSi1
#data clean:
df['light_dark'] = np.where(df.roast=='Very Dark','Dark',df.roast)
df = df.loc[(df.light_dark=='Dark')|(df.light_dark=='Light')]
df = df.loc[(df.blind_assesment.str.len()>10)]
df['y'] = np.where(df.light_dark=='Light',1,0)
df.light_dark.value_counts()
df.columns

In [ ]:
X = df.blind_assesment.values
y = df.y.values

tokenizer = Tokenizer(num_words=4000)
tokenizer.fit_on_texts(X)

X = tokenizer.texts_to_sequences(X)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

maxlen = 200
embedding_dim = 50

X = pad_sequences(X, padding='post', maxlen=maxlen)


sequence_input = layers.Input(shape=(maxlen,), dtype='int32')
embedded_sequences = layers.Embedding(vocab_size, embedding_dim, input_length=maxlen)(sequence_input)
l_cov1  = layers.Conv1D(317, 3, activation='relu')(embedded_sequences)
l_pool1 = layers.MaxPooling1D(2)(l_cov1)
l_cov2  = layers.Conv1D(317, 1, activation='relu')(l_pool1)
l_cov3  = layers.Conv1D(317, 2, activation='relu')(l_cov2)
l_pool3 = layers.GlobalMaxPooling1D()(l_cov3)  # global max pooling
l_bnorm = layers.BatchNormalization()(l_pool3)
l_dense = layers.Dense(128, activation='relu')(l_pool3)
preds   = layers.Dense(1, activation='sigmoid',name='preds')(l_dense)


model = Model(sequence_input, outputs=preds)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

model.fit(X, y, epochs=3, validation_split=0.1, batch_size=10)

In [ ]:
#@title ## Coffee darkness predictor
#@markdown Try-it-out by pasting or typing your coffee taste description below:
type_here = ""#@param {type:"string"}
maxlen = 200
Xtst = tokenizer.texts_to_sequences([type_here])
Xtst = pad_sequences(Xtst, padding='post', maxlen=maxlen)

y_pred = model.predict(Xtst)

class_idx = np.argmax(y_pred[0]) #not needed in this case as only two classes
class_output = model.output[:, class_idx]
last_conv_layer = model.get_layer("conv1d_1")

grads = K.gradients(class_output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads)
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([Xtst])

    
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap,0)
heatmap /= np.max(heatmap)#normalise values in the prediction

norm_len = maxlen/last_conv_layer.output_shape[1] # find the ratio of the text vs the conv layer length

html = ""
if y_pred[0]>0.5:
  pred = 'light'
else:
  pred = 'dark'
html += "<span><h3>Based on the description, the model believes that this is a {} coffee roast. ".format(pred)
html += "<small><br>Confidence: {:.0f}%<br><br></small></h3></span>".format(abs(((y_pred[0][0]*100)-50)*2))
for j,i in enumerate(tokenizer.sequences_to_texts(Xtst)[0].split()):
  html += "<span style='background-color:rgba({},0,150,{})'>{} </span>".format(heatmap[math.floor(j/norm_len)]*255,heatmap[math.floor(j/norm_len)]-0.3,i)

HTML(html)
